In [86]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D,Flatten, Dropout
from keras.applications import MobileNet,MobileNetV2,VGG16
from keras.applications.mobilenet import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

In [87]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [88]:
def Save_Model(model):
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model.h5")
    print("Saved model to disk")

In [89]:
base_model = VGG16(include_top=False,weights='imagenet') 
base_model.trainable = False
x=base_model.output
x=GlobalAveragePooling2D()(x)
x = Flatten()(x)
x=Dense(1024,activation='relu')(x)
#x = Dropout(0.2)(x)
#x=Dense(1024,activation='relu')(x)
#x = Dropout(0.2)(x)
preds=Dense(12,activation='softmax')(x) 

In [90]:
model=Model(inputs=base_model.input,outputs=preds)

In [91]:
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0  

In [92]:

for i,layer in enumerate(model.layers):
  print(i,layer.name)


0 input_20
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_18
20 flatten_17
21 dense_41
22 dense_42


In [93]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.15) 
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.15) 

train_generator = train_datagen.flow_from_directory(
        r'C:\Users\mhfar\Desktop\Trunk\Data',
        color_mode="rgb",
         target_size=(224,224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False,
        seed=42,
        subset = 'training')

test_data = test_datagen.flow_from_directory(
        r'C:\Users\mhfar\Desktop\Trunk\Data',
        color_mode="rgb",
         target_size=(224,224),
        batch_size=32,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        subset = 'validation')

Found 341 images belonging to 12 classes.
Found 52 images belonging to 12 classes.


In [94]:

es = EarlyStopping(monitor='val_accuracy',
                    mode='max',
                    patience=99,
                    verbose=1,
                    restore_best_weights=True,
                    baseline= 1)


opt = keras.optimizers.Adam(learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,)

#opt = SGD(learning_rate= 0.001)

model.compile(optimizer=opt ,loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
model.fit(train_generator,
                validation_data=test_data,
                callbacks=[es],
                verbose=1,
                steps_per_epoch=step_size_train,
                epochs=30)

Epoch 1/30
10/10 [==============================] - 54s 5s/step - loss: 3.3907 - accuracy: 0.0000e+00 - val_loss: 2.4477 - val_accuracy: 0.1154
Epoch 2/30
10/10 [==============================] - 49s 5s/step - loss: 2.6598 - accuracy: 0.1068 - val_loss: 2.2306 - val_accuracy: 0.4423
Epoch 3/30
10/10 [==============================] - 47s 5s/step - loss: 2.2394 - accuracy: 0.3236 - val_loss: 1.9527 - val_accuracy: 0.3269
Epoch 4/30
10/10 [==============================] - 48s 5s/step - loss: 2.0748 - accuracy: 0.3139 - val_loss: 1.7502 - val_accuracy: 0.5192
Epoch 5/30
10/10 [==============================] - 48s 5s/step - loss: 1.7873 - accuracy: 0.4207 - val_loss: 1.4799 - val_accuracy: 0.7115
Epoch 6/30
10/10 [==============================] - 50s 5s/step - loss: 1.5922 - accuracy: 0.5437 - val_loss: 1.3476 - val_accuracy: 0.6923
Epoch 7/30
10/10 [==============================] - 50s 5s/step - loss: 1.3252 - accuracy: 0.6699 - val_loss: 1.2121 - val_accuracy: 0.7692
Epoch 8/30
10/10

In [95]:
model.evaluate(train_generator, verbose=2)

11/11 - 45s - loss: 0.3036 - accuracy: 0.9384 - 45s/epoch - 4s/step


[0.30363309383392334, 0.9384164214134216]

In [96]:
model.evaluate(test_data, verbose=2)

2/2 - 7s - loss: 0.3669 - accuracy: 0.9038 - 7s/epoch - 4s/step


[0.36693185567855835, 0.9038461446762085]

In [97]:
Save_Model(model)

Saved model to disk
